In [ ]:
!pip install pandas

In [ ]:
import pandas
import numpy
import s3fs
import os
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType
from pyspark.sql.functions import rand

In [ ]:
# Génération de données simulées
data = {'identifiant': [str(i) for i in range(1, 500001)]}

df = pandas.DataFrame(data)

In [ ]:
parquet_file = 'identifiants.parquet'  # Chemin local où le fichier Parquet sera temporairement stocké
df.to_parquet(parquet_file, index=False)

In [ ]:
!mc cp identifiants.parquet s3/auredisanto/data-exemples-avantages-spark/identifiants.parquet

In [ ]:
# Suppression du fichier local temporaire
os.remove(parquet_file)

In [ ]:
spark = (SparkSession 
         .builder
         .getOrCreate()
        )

sc = spark.sparkContext

In [ ]:
parquet_df = spark.read.parquet("s3a://auredisanto/data-exemples-avantages-spark/identifiants.parquet")
parquet_df.createOrReplaceTempView("identifiants")

In [ ]:
spark_df1 = spark.sql("SELECT *, rand() as col1_fichier1, rand() as col2_fichier1 FROM identifiants") 
spark_df1.write.mode('append').parquet("s3a://auredisanto/data-exemples-avantages-spark/fichier1.parquet")

In [ ]:
spark_df2 = spark.sql("SELECT *, rand() as col1_fichier2, rand() as col2_fichier2 FROM identifiants") 
spark_df2.write.mode('append').parquet("s3a://auredisanto/data-exemples-avantages-spark/fichier2.parquet")

In [ ]:
spark_df3 = spark.sql("SELECT *, rand() as col1_fichier3, rand() as col2_fichier3 FROM identifiants") 
spark_df3.write.mode('append').parquet("s3a://auredisanto/data-exemples-avantages-spark/fichier3.parquet")

In [5]:
spark.stop()